# 📘 Clase 15: pytest Avanzado

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/heldigard/unaula-IF0100-POO-II/blob/main/notebooks/unidad-02/clase-15-pytest-avanzado.ipynb)

## 🎯 Objetivos de Aprendizaje

Al finalizar esta clase, seras capaz de:
- Usar fixtures para preparacion de tests
- Crear mocks con unittest.mock
- Parametrizar tests
- Organizar tests con marcas (marks)
- Medir cobertura de codigo

---

## 📚 Fixtures

In [ ]:
# ============================================
# FIXTURES EN pytest
# ============================================

import pytest

# Fixture simple
@pytest.fixture
def lista_numeros():
    return [1, 2, 3, 4, 5]

# Fixture con setup y teardown
@pytest.fixture
def archivo_temporal():
    # Setup
    import tempfile
    archivo = tempfile.NamedTemporaryFile(mode='w', delete=False)
    archivo.write('contenido de prueba')
    archivo.close()
    
    yield archivo.name  # Retorna valor a los tests
    
    # Teardown (limpieza)
    import os
    os.unlink(archivo.name)

# Fixture con scope (session, module, class, function)
@pytest.fixture(scope='module')
def base_de_datos():
    """Simula conexion a BD que se reutiliza por modulo."""
    db = {'usuarios': [], 'conectado': True}
    yield db
    db['conectado'] = False

# Demostracion (sin ejecutar pytest)
print('✅ Fixtures definidos')
print('- lista_numeros: Fixture simple')
print('- archivo_temporal: Fixture con setup/teardown')
print('- base_de_datos: Fixture con scope module')

In [ ]:
# ============================================
# USANDO FIXTURES EN TESTS
# ============================================

class Usuario:
    def __init__(self, username, email):
        self.username = username
        self.email = email
        self.activo = True
        self.proyectos = []
    
    def agregar_proyecto(self, proyecto):
        self.proyectos.append(proyecto)
    
    def desactivar(self):
        self.activo = False

class Proyecto:
    def __init__(self, nombre, creador):
        self.nombre = nombre
        self.creador = creador
        self.tareas = []
    
    def agregar_tarea(self, tarea):
        self.tareas.append(tarea)

# Fixtures para nuestros modelos
def usuario_fixture():
    return Usuario('testuser', 'test@example.com')

def proyecto_fixture(usuario):
    return Proyecto('Proyecto Test', usuario)

# Tests usando fixtures manualmente (para demostracion)
def test_usuario_creacion():
    usuario = usuario_fixture()
    assert usuario.username == 'testuser'
    assert usuario.activo == True
    print('✅ test_usuario_creacion PASA')

def test_proyecto_tiene_creador():
    usuario = usuario_fixture()
    proyecto = proyecto_fixture(usuario)
    assert proyecto.creador == usuario
    assert proyecto.nombre == 'Proyecto Test'
    print('✅ test_proyecto_tiene_creador PASA')

test_usuario_creacion()
test_proyecto_tiene_creador()

---

## 🎭 Mocks

In [ ]:
# ============================================
# MOCKING CON unittest.mock
# ============================================

from unittest.mock import Mock, patch, MagicMock

# Mock simple
mock = Mock()
mock.metodo.return_value = 42
print(f'Mock retorna: {mock.metodo()}')

# Verificar llamadas
mock.otro(1, 2, key='value')
print(f'Fue llamado: {mock.otro.called}')
print(f'Con argumentos: {mock.otro.call_args}')

# Patch de funciones
import random

def funcion_que_usa_random():
    return random.randint(1, 100)

# Simular que random siempre retorna 42
with patch('random.randint', return_value=42):
    resultado = funcion_que_usa_random()
    print(f'\nCon mock: {resultado}')

# Sin mock (valor real)
print(f'Sin mock: {funcion_que_usa_random()}')

In [ ]:
# ============================================
# EJEMPLO PRACTICO DE MOCKING
# ============================================

class ServicioEmail:
    """Servicio real (lento, requiere conexion)."""
    def enviar(self, destinatario, mensaje):
        # Simula envio real
        import time
        time.sleep(2)  # Lento!
        return f'Email enviado a {destinatario}'

class Notificador:
    """Usa el servicio de email."""
    def __init__(self, servicio_email):
        self.email = servicio_email
    
    def notificar_bienvenida(self, usuario):
        mensaje = f'Bienvenido, {usuario}!'
        return self.email.enviar(usuario, mensaje)

# Test con mock (rapido, no requiere conexion)
def test_notificar_bienvenida():
    # Crear mock del servicio
    mock_email = Mock()
    mock_email.enviar.return_value = 'Email enviado (mock)'
    
    # Inyectar mock
    notificador = Notificador(mock_email)
    resultado = notificador.notificar_bienvenida('juan@test.com')
    
    # Verificar
    assert resultado == 'Email enviado (mock)'
    mock_email.enviar.assert_called_once_with(
        'juan@test.com', 
        'Bienvenido, juan@test.com!'
    )
    print('✅ Test con mock PASA (rapido, aislado)')

test_notificar_bienvenida()

---

## 📊 Parametrizacion

In [ ]:
# ============================================
# TESTS PARAMETRIZADOS
# ============================================

# Sin parametrizar (repetitivo)
def test_es_mayor_de_edad_18():
    assert es_mayor_de_edad(18) == True

def test_es_mayor_de_edad_17():
    assert es_mayor_de_edad(17) == False

def test_es_mayor_de_edad_65():
    assert es_mayor_de_edad(65) == True

# Con parametrizacion (mas limpio)
# En pytest seria:
# @pytest.mark.parametrize('edad,esperado', [
#     (18, True),
#     (17, False),
#     (65, True),
#     (0, False),
# ])
# def test_es_mayor_de_edad(edad, esperado):
#     assert es_mayor_de_edad(edad) == esperado

def es_mayor_de_edad(edad):
    return edad >= 18

# Simulacion manual de parametrizacion
casos = [
    (18, True),
    (17, False),
    (65, True),
    (0, False),
    (100, True),
]

print('=== Tests Parametrizados ===')
for edad, esperado in casos:
    resultado = es_mayor_de_edad(edad) == esperado
    status = '✅' if resultado else '❌'
    print(f'{status} es_mayor_de_edad({edad}) == {esperado}')

---

## 📝 Ejercicios

### Ejercicio 1: Test con Fixtures y Mocks
Crea tests para un servicio de autenticacion usando mocks para la base de datos.

In [ ]:
# Ejercicio 1: Servicio de Autenticacion

class ServicioAutenticacion:
    def __init__(self, db):
        self.db = db
    
    def autenticar(self, username, password):
        usuario = self.db.buscar_usuario(username)
        if not usuario:
            return None
        if usuario['password'] == password:
            return usuario
        return None
    
    def registrar(self, username, password):
        if self.db.buscar_usuario(username):
            raise ValueError('Usuario ya existe')
        self.db.guardar_usuario({'username': username, 'password': password})
        return True

# Escribe tests usando mocks
# def test_autenticar_exitoso():
#     # Mock db que retorna usuario
#     pass

# def test_autenticar_fallido():
#     # Mock db que retorna None
#     pass

---

## 🔗 Conexion con TaskFlow

En TaskFlow usamos fixtures compartidas en conftest.py:

```python
# tests/conftest.py
import pytest

@pytest.fixture
def cliente():
    from taskflow.api.app import app
    from fastapi.testclient import TestClient
    return TestClient(app)

@pytest.fixture
def usuario_data():
    return {
        'username': 'testuser',
        'email': 'test@example.com',
        'password': 'secret123'
    }
```

---

**¡Aísla, simula, verifica! 🎭**